# Phase IV: Storing and Sharing Your Research-Ready Database

You have done a lot of heavy lifting and learned a lot. And you have a really useful set of data to show for it. Now the question becomes: how do you want tostore and share all this data? 

Throughout these tutorials, we have guided you step-by-step through the processes and code necessary to make your database research-ready. Here, our goals are more limited. Because there are dozens (or more) ways you could store and share your data, we aren't able to show you all of them. Instead, we will focus on a brief introduction into our choice, namely storing the data in an SQLite database and sharing it via a python server using a library called Flask. Again, this is just one approach that we found practical. It may also be applicable for many of you though certainly not all of you.

## 1. Creating a SQLite database

In the last phase you learned to spread out your data into a series of tables. Each table with a more managable size than the alternative of storing it all in one gigantic table. One tool to manage such a relational database system is SQLite. SQLite is based on the omnipresent SQL language but optimized for use in (relatively) small environments.

There are two ways to store our freshly-parsed text into an SQLite database that we will introduce here. The first is writing directly into an ever-growing SQLite database as you iterate through your documents. The second solution is to create CSV files as an intermediate storage, and only after completing the iterative parsing convert the completed CSV files into an SQLite database.

### Option A: Extending the database as you iterate through the documents

To build your database, you could use SQLite's commands to store your parsed texts at the end of parsing each document. 

To do this, we first have to create a database that we can later write into. In our simple example here, we will create a database with one table only, called 'hearing'. We send all our commands using an object called 'cursor', the controller we can use to manipulate our database.

Note that the commands insided the brackets of ``cursor.execute('...')`` are written in SQL. Click <a href="https://www.w3schools.com/sql/" target="_blank">here</a> for a very good hands-on SQL tutorial.

We have now created our simple SQLite databse including a rudimentary 'hearing' table. The plan now is to fill it consecutively with every document as soon as it has been parsed. 

To do that, we would go to the code position at the end of a document loop. There we add a few commands to reconnect to our SQLite database, take what has been found in the freshly-parsed document and add it to the hearing table. Before we turn to the next document, we store the updated datbase and close the connection again.

The code passage would looks similar to what follows:

###  Option B: Store in CSV first, then convert it all in one go [Recommended]

To see where this is going, let's have a look at a few sample CSV files that include text as parsed in Phase III.

We have four files and you can think of them as four separate tables inside our relational database.

Hearing

Speech

Speaker

Person

In [ ]:
import pandas as pd
hearings = pd.read_csv("data/hearing.csv")
hearings.head()

In [ ]:
speech = pd.read_csv("data/speech.csv")
speech.head()

In [ ]:
speaker = pd.read_csv("data/speaker.csv")
speaker.head()

In [ ]:
person = pd.read_csv("data/person.csv")
person.head()

#### Creating CSV files
But how did we get to these CSVs in the first place. Again, there are various options. Our preferred one uses python's 'datascience' library (install it typing 'pip install datascience' into your terminal, if you have not done so already).

Using the datascience library we can create a CSV file in three steps.

In [ ]:
import datascience as ds

### Create a table with the desired column names
columns = ("hearing_id", "committee_id", "subcommittee_id","hearing_title", "is_appropriation", "is_nomination", "date","url","file","extent")
hearing_table = ds.Table(columns)

### Add data to the table. Direct your parser to store data as an array and them add that array into your table.
row = ["1", "1", "1", "Example hearing", "1", "0", "2018/01/01", "someurl.gov", None, None]
hearing_table = hearing_table.with_row(row)

### Save the data as a csv file
hearing_table.to_csv("sample/hearing_csv_made_using_ds.csv")

Now read the file and see that we've added a new row into the csv file:

In [ ]:
person = pd.read_csv("sample/hearing_csv_made_using_ds.csv")
person.head()

#### Quiz: What happens when you add an empty array to the table?

In [ ]:
### Enter your answer here.

#### Now you have your CSV files, what next?

It's now time to create your SQLite database. Here, we will be using python library called 'pandas' to help us do the conversion.

Before pandas can write into our database, we have to reconnect to our database first.

In [ ]:
import sqlite3
import pandas as pd

connection = sqlite3.connect("mydatabase.sqlite")

### Loading our table CSVs
hearing = pd.read_csv("data/hearing.csv")
speech = pd.read_csv("data/speech.csv")
speaker = pd.read_csv("data/speaker.csv")
person = pd.read_csv("data/person.csv")

### Converting each table to SQL using the 'connection', 
### appending (rather than replacing) existing data and supplying the indexing ID (instead of having panda generate a new one)
hearing.to_sql("hearing", connection, if_exists='append', index=False)
speech.to_sql("speech", connection, if_exists='append', index=False)
speaker.to_sql("speaker", connection, if_exists='append', index=False)
person.to_sql("person", connection, if_exists='append', index=False)

Now let's read the contents we have added. Let's do this with a simply query. How many speakers are there?

In [ ]:
cursor = connection.cursor()

cursor.execute("SELECT count(*) FROM speaker;")

no_of_speakers = cursor.fetchall()

print(no_of_speakers)


#### Quiz: How many hearings are there?

In [ ]:
### Try it here.

In [ ]:
### Always a good idea to close a connection when done.
connection.close()

# 2. Sharing your data online: Flask

Flask is a lightweight framework for writing web applications. It is called 'lightweight' because it has little to no dependency on external libraries for it to function. You write the code in python and you can run your apps locally or on the server. To install Flask, run the following on your terminal:

## Let's create a simple Flask app

When coding and running Flask, you can no longer use jupyter notebooks. So for this section, follow along the instructions to see how you can run an example of Flask.

Using a text editor, open the app.py file in the root folder of our tutorial. There, you'll see the following code.

In [ ]:
#!/usr/bin/env python

import flask

# Create the application.
app = flask.Flask(__name__)

@app.route('/')
def index():
    return flask.render_template('index.html')


if __name__ == '__main__':
    app.debug=True
    app.run()

Using your terminal, you can run this app by typing:

Then load 'http://127.0.0.1:5000/' on your browser where you will see rudimentary webapp.

## Let's pull some data from the DB and access it from the browser

Add this to the top of the app.py code:

In [ ]:
from flask import Response
import sqlite3
import json

Add this in the body of the text:

In [ ]:
@app.route('/speakers')
def speakers():
    connection = sqlite3.connect("mydatabase.sqlite") 
    cursor = connection.cursor()
    cursor.execute("SELECT surname FROM speaker;")
    speakers = cursor.fetchall()
    return Response(json.dumps(speakers), mimetype='application/json')

Now restart your app and see a rudimentary search function for this basic data.

You already saw how little coding is necessary to get started. To move one, we recommend looking at <a href="https://pythonspot.com/flask-web-app-with-python/" target="_blank">this beginner's introduction</a>. Add in some HTML, or even <a href="https://bokeh.pydata.org/" target="_blank">Bokeh</a>, and things get pretty (as well as) useful very fast.